In [11]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn import datasets, metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier # RandomForestClassifier: Um classificador baseado em uma floresta aleatória (Random Forest), que utiliza múltiplas árvores de 
from sklearn.model_selection import GridSearchCV

# Carregando o dataset Iris
iris = datasets.load_iris()

# Extraindo os dados e criando um DataFrame
X = iris.data  # Características (features)
y = iris.target  # Classes (target)
feature_names = iris.feature_names
target_names = iris.target_names

# Criando um DataFrame para melhor visualização
df = pd.DataFrame(X, columns=feature_names)
df['species'] = [target_names[i] for i in y]

# Visualizando as primeiras linhas do dataset
print("Primeiras linhas do dataset Iris:")
print(df.head())

Primeiras linhas do dataset Iris:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

  species  
0  setosa  
1  setosa  
2  setosa  
3  setosa  
4  setosa  


In [12]:
# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)  
    # test_size=0.25: Define que 25% dos dados serão usados para o conjunto de teste.
    # random_state=42: Este parâmetro garante que a divisão dos dados seja reproduzível.
    # X_train terá 75% dos dados, enquanto X_test terá 25%.
    # y_train terá os rótulos para os 75% dos dados de treino, enquanto y_test terá os rótulos para os 25% dos dados de teste.


In [13]:
scaler = StandardScaler() # Cria um objeto para escalonar os dados. 
# StandardScaler: Usado para escalonamento (normalização) das features, transformando os dados para que tenham média zero e desvio padrão um. 
X_train_scaled = scaler.fit_transform(X_train) # aplica o ajuste e a transformação de uma vez.
# OneHotEncoder: Codifica variáveis categóricas em uma forma que o modelo possa entender.
X_test_scaled = scaler.transform(X_test)
# SimpleImputer: Utilizado para lidar com valores ausentes nos dados (missing values).

In [22]:
# Exemplo mais complexo com diferentes tipos de dados
# (normalmente usado quando temos features numéricas e categóricas)
numeric_features = [0, 1, 2, 3]  # índices das colunas numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
# ColumnTransformer: Permite aplicar transformações diferentes em colunas específicas do conjunto de dados.
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

In [15]:
# Pipeline simples com pré-processamento e modelo
pipeline = Pipeline(steps=[
    # Pipeline é uma forma de encapsular um fluxo completo de transformação dos dados e treinamento de um modelo em uma única estrutura.
    ('preprocessor', preprocessor), # Ele aplica as transformações definidas anteriormente no ColumnTransformer
    ('classifier', RandomForestClassifier(random_state=42)) # É o modelo de aprendizado de máquina que será treinado. Aqui, está sendo utilizado o RandomForestClassifier
])


In [16]:
# Treinar o pipeline
pipeline.fit(X_train, y_train)   
# O preprocessor aplica as transformações (como imputação e escalonamento) nas features de treinamento (X_train).

# Fazer previsões
y_pred = pipeline.predict(X_test) 
# Após o treinamento, o pipeline é capaz de fazer previsões nos dados de teste (X_test), aplicando o mesmo pré-processamento aos dados de entrada antes de fazer as previsões com o modelo.

In [17]:
# Avaliar a precisão 
accuracy = metrics.accuracy_score(y_test, y_pred) 
print(f"Acurácia: {accuracy:.4f}") 

# Relatório de classificação 
print("Relatório de classificação:") 
print(metrics.classification_report(y_test, y_pred, 
                                   target_names=iris.target_names)) 

# Matriz de confusão 
cm = metrics.confusion_matrix(y_test, y_pred) 
print("Matriz de confusão:") 
print(cm) 


Acurácia: 1.0000
Relatório de classificação:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       1.00      1.00      1.00        11
   virginica       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

Matriz de confusão:
[[15  0  0]
 [ 0 11  0]
 [ 0  0 12]]


In [24]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Acurácia:", accuracy)
print("Matriz de Confusão:\n", conf_matrix)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

Acurácia: 1.0
Matriz de Confusão:
 [[15  0  0]
 [ 0 11  0]
 [ 0  0 12]]
Precisão: 1.0
Recall: 1.0
F1-Score: 1.0


In [25]:

# Definir parâmetros para busca
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30]
}

# Criar objeto de busca em grade
grid_search = GridSearchCV(
    pipeline, param_grid, cv=5, scoring='accuracy'
)

# Executar busca
grid_search.fit(X_train, y_train)

# Melhores parâmetros
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação:", grid_search.best_score_)

# Usar o melhor modelo
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
print(f"Acurácia com o melhor modelo: {metrics.accuracy_score(y_test, y_pred_best):.4f}")

Melhores parâmetros: {'classifier__max_depth': None, 'classifier__n_estimators': 200}
Melhor pontuação: 0.9462450592885375
Acurácia com o melhor modelo: 1.0000
